In [ ]:
# Importing Required Libraries
import os
import pandas as pd
import mysql.connector

In [ ]:
# MySQL database configuration
# MYSQL_HOST = "localhost"
# MYSQL_USER = "root"
# MYSQL_PASSWORD = "Password!!"
MYSQL_HOST = "localhost"
MYSQL_USER = "mh30f"
MYSQL_PASSWORD = "Moh@1977"

In [ ]:
 # function ensures dates are converted to YYYY-MM-DD format. 
def convert_date_format(date_str):
    if isinstance(date_str, str) and date_str != '0':
        parts = date_str.split('/')
        if len(parts) == 3:
            if len(parts[0]) == 4:  # YYYY/MM/DD
                return '-'.join(parts)
            else:  # DD/MM/YYYY
                return '-'.join(parts[::-1])
    return '0000-00-00'

In [ ]:
# The main() function handles the core logic of connecting to MySQL, processing CSV files, and inserting data.
def main():
    try:
        # Connect to MySQL
        connection = mysql.connector.connect(
            host=MYSQL_HOST,
            user=MYSQL_USER,
            password=MYSQL_PASSWORD,
            auth_plugin="mysql_native_password"
        )
        cursor = connection.cursor()

        # Get list of CSV files in current directory
        # files = [f for f in os.listdir('.') if f.endswith('.csv')]
        resources_dir = os.path.join(os.path.dirname(os.path.abspath(__file__)), 'Resources')
        files = [f for f in os.listdir(resources_dir) if f.endswith('.csv')]

        for file_name in files:
            db_name = os.path.basename(os.path.dirname(os.path.abspath(__file__)))
            table_name = os.path.splitext(file_name)[0]

            # Read CSV into DataFrame
            df = pd.read_csv(file_name, skiprows=[1, 2])
            df.columns = df.columns.str.strip().str.replace(' ', '_')

            if 'Date' in df.columns and 'Time' in df.columns:
                df['Time'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], errors='coerce', dayfirst=True)
                df.drop(columns=['Date'], inplace=True)
                df['Time'] = df['Time'].apply(convert_date_format)

            # Create database and table
            cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
            cursor.execute(f"USE {db_name}")
            
            create_table_query = "CREATE TABLE IF NOT EXISTS {table_name} ("
            for col in df.columns:
                dtype = 'float'
                if col == 'Time':
                    dtype = 'datetime DEFAULT NULL'
                elif pd.api.types.is_integer_dtype(df[col]):
                    dtype = 'int'
                elif pd.api.types.is_string_dtype(df[col]):
                    dtype = 'varchar(255)'
                create_table_query += f"`{col}` {dtype}, "
            create_table_query = create_table_query.rstrip(', ') + ")"
            cursor.execute(create_table_query)

            # Batch insert rows
            rows = [tuple(row) for row in df.itertuples(index=False, name=None)]
            placeholders = ', '.join(['%s'] * len(df.columns))
            insert_query = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({placeholders})"
            cursor.executemany(insert_query, rows)

        connection.commit()

    except mysql.connector.Error

In [ ]:
if __name__ == "__main__":
    main()